Imports

In [1]:
import pygad
import random
from read_data import read_dataset_1, translate_1
from models import SimulationEnvironment

Parameters

In [2]:
order_amount = 5
earliest_time_slot = 0
last_time_slot = 5000 # shouldn't actually be necessary for this

Input

In [3]:
input, orders, instance = read_dataset_1(use_instance=13, order_amount=order_amount, earliest_time=earliest_time_slot, last_time=last_time_slot)
recipes, workstations, resources, tasks, orders_model = translate_1(instance, orders, earliest_time_slot, last_time_slot)
env = SimulationEnvironment(workstations, tasks, resources, recipes)

Setup

In [4]:
assignments = []
start_times = []

duration_lookup_table = dict()
for task in tasks:
    for workstation in env.get_valid_workstations(task.id):
        if not task in duration_lookup_table:
            duration_lookup_table[task.id] = dict()
        duration_lookup_table[task.id][workstation.id] = env.get_duration(task.id, workstation.id)

operations = []
order_for_index = []
for order in orders_model:
    for resource in order.resources:
        recipe = resource.recipes[0] # just use recipe 0 for now
        recipe_tasks = env.get_all_tasks_for_recipe(recipe.id)
        results = dict()
        for task in recipe_tasks:
            if task.result_resources[0][0] not in results:
                results[task.result_resources[0][0]] = []
            results[task.result_resources[0][0]].append(task)
        for key in results:
            operations.append(random.choice(results[key]).id)
            order_for_index.append(order.id)

for operation in operations:
    workstation = env.get_valid_workstations(operation)
    # random init
    assignments.append(random.choice(workstation).id)
    assignments.append(0) # start time slot

Helper Functions

In [ ]:
def calculate_start_time(individual):
    i = 0
    operation_index = 0
    for gene in individual:
        if i == 1:
            on_workstation = [] # find all assignments to the same workstation
            prev_operations = [] # find all operations belonging to the same order, which need to be scheduled before the current operation
            # calculate start time, aka choose max(last end time on workstation, last end time of previous sequenced operation for task)
            pass
        i += 1
        if i > 1:
            i = 0
            operation_index += 1
    return individual

GA Functions

In [ ]:
def mutation_function(offspring, ga_instance):
    i = 0
    operation_index = 0
    p = ga_instance.mutation_percent_genes
    for gene in offspring:
        if i == 0:
            if random.random() < p:
                # mutate
                # according to paper, calc workload of each elligible workstation, switch to lowest workload
                pass
        i+=1
        if i > 1:
            i = 0
            operation_index += 1
    # re-calculate start times
    calculate_start_time(offspring)
    return offspring

# make sure crossover is performed at workstation assignments
def crossover_function(parents, offspring_size, ga_instance):
    split_point = random.randint(0, offspring_size)
    if split_point % 2 == 1:
        split_point -= 1
    parent1 = parents[0].copy()
    parent2 = parents[1].copy()
    offspring = []
    for i in range(offspring_size):
        if i < split_point:
            offspring.append(parent1[i])
        else:
            offspring.append(parent2[i])
    return offspring

def fitness_function(solution, solution_idx):
    fitness = 1
    i = 0
    operation_index = 0
    max = -float('inf')
    min = float('inf')
    for idx in range(len(solution)):
        if i == 1:
            start = solution[idx]
            end = start + duration_lookup_table[operations[idx]][solution[idx-1]] # double check operations
            if start < min:
                min = start
            if end > max:
                max = end
        i += 1
        if i > 1:
            i = 0
            operation_index += 1
    fitness = abs(max - min)
    return -fitness


Run

In [ ]:
num_genes = len(assignments)
num_generations = 5000
num_parents_mating = 50
sol_per_pop = 100
init_range_low = 0
init_range_high = last_time_slot
parent_selection_type = 'rws'
keep_parents = 10
crossover_type = crossover_function
mutation_type = mutation_function
mutation_percentage_genes = 0.1
fitness_func = fitness_function
gene_type = int
space_workstations = {'low': 1, 'high': len(workstations)-1}
space_time = {'low': 0, 'high': last_time_slot}
gene_space = []
i = 0
for _ in range(len(assignments)):
    if i == 0:
        gene_space.append(space_workstations)
    else:
        gene_space.append(space_time) # shouldn't be needed because of manual mutation and crossover
    i += 1
    if i > 1:
        i = 0
ga_instance = pygad.GA(num_generations=num_generations, num_parents_mating=num_parents_mating, fitness_func=fitness_func, sol_per_pop=sol_per_pop, num_genes=num_genes, init_range_low=init_range_low, init_range_high=init_range_high, parent_selection_type=parent_selection_type, keep_parents=keep_parents, crossover_type=crossover_type, mutation_type=mutation_type, mutation_percent_genes=mutation_percentage_genes, gene_type=gene_type, gene_space=gene_space)
ga_instance.run()
solution, solution_fitness, solution_idx = ga_instance.best_solution()

print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=abs(solution_fitness) - 1))